In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Load the dataset from Excel
df = pd.read_excel('IMDB Dataset.xlsx')

# Extracting features (movie reviews) and labels (sentiments)
X = df['review']
y = df['sentiment']

# Encode labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Splitting the dataset into training and testing sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenizing the text data
max_words = 10000  # maximum number of words to keep
max_len = 200  # maximum length of a sequence
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequence = tokenizer.texts_to_sequences(X_train)
X_test_sequence = tokenizer.texts_to_sequences(X_test)

# Padding sequences to ensure uniform length
X_train_padded = pad_sequences(X_train_sequence, maxlen=max_len)
X_test_padded = pad_sequences(X_test_sequence, maxlen=max_len)

# Defining the CNN model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_len))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training the model
model.fit(X_train_padded, y_train, batch_size=128, epochs=5, validation_split=0.1)

# Evaluating the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/5
282/282 [==============================] - 40s 122ms/step - loss: 0.4607 - accuracy: 0.7675 - val_loss: 0.2753 - val_accuracy: 0.8827
Epoch 2/5
282/282 [==============================] - 18s 64ms/step - loss: 0.2246 - accuracy: 0.9149 - val_loss: 0.2396 - val_accuracy: 0.9018
Epoch 3/5
282/282 [==============================] - 10s 36ms/step - loss: 0.1094 - accuracy: 0.9661 - val_loss: 0.2691 - val_accuracy: 0.8965
Epoch 4/5
282/282 [==============================] - 7s 25ms/step - loss: 0.0426 - accuracy: 0.9900 - val_loss: 0.3035 - val_accuracy: 0.8992
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.3755 - accuracy: 0.8949
Test Accuracy: 0.8949000239372253


In [13]:
# Assume 'input_review' is the new review you want to predict
input_review = input("Type your Review: ")

# Tokenize and pad the input review
input_review_sequence = tokenizer.texts_to_sequences([input_review])
input_review_padded = pad_sequences(input_review_sequence, maxlen=max_len, padding='post')

# Make prediction
prediction = model.predict(input_review_padded)

# Convert prediction to sentiment label
predicted_sentiment = "positive" if prediction[0][0] > 0.5 else "negative"

print("Predicted Sentiment:", predicted_sentiment)

Type your Review: Poor movie
1/1 [==============================] - 0s 17ms/step
Predicted Sentiment: negative
